In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
DEBUG=False

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
nmodel_dir = '/home/ashesh.ashesh/training/noise_model/2403/199'
# nmodel_dir = '/home/ashesh.ashesh/training/noise_model/2402/61'

histnoisemodel_fpath = None
gmmnoisemodel_fpath = None
for fname in os.listdir(nmodel_dir):
    if fname.startswith('HistNoiseModel'):
        assert histnoisemodel_fpath is None
        histnoisemodel_fpath = os.path.join(nmodel_dir, fname)
    elif fname.startswith('GMMNoiseModel'):
        assert gmmnoisemodel_fpath is None
        gmmnoisemodel_fpath = os.path.join(nmodel_dir, fname)
print(gmmnoisemodel_fpath)
print(histnoisemodel_fpath)

In [ ]:
from denoisplit.nets.gmm_noise_model import GaussianMixtureNoiseModel
from denoisplit.nets.hist_noise_model import HistNoiseModel

# gmmnoisemodel_fpath = '/home/ashesh.ashesh/training/noise_model/2402/62/GMMNoiseModel_CCPs-GT_all.mrc__6_4_Clip0.0-1.0_Sig0.125_UpNone_Norm0_bootstrap.npz'
# histnoisemodel_fpath = os.path.join(os.path.dirname(gmmnoisemodel_fpath), 'HistNoiseModel_CCPs-GT_all.mrc__Norm0_Bins128_bootstrap.npy')
# datadir = '/group/jug/ashesh/data/ventura_gigascience/actin-60x-noise2-highsnr.tif' if 'actin' in os.path.basename(gmmnoisemodel_fpath) else '/group/jug/ashesh/data/ventura_gigascience/mito-60x-noise2-highsnr.tif'


In [ ]:
nmodel_params = np.load(gmmnoisemodel_fpath)
gmm_model = GaussianMixtureNoiseModel(params=nmodel_params)
histdata = np.load(histnoisemodel_fpath)
hist_model = HistNoiseModel(histdata)
bins = histdata.shape[-1]

In [ ]:
histdata[1,25:50,0]

In [ ]:
plt.imshow(histdata[0])

In [ ]:
from denoisplit.utils import plotProbabilityDistribution
signalBinIndex= 40
data_dict = plotProbabilityDistribution(signalBinIndex=signalBinIndex, 
                            histogramNoiseModel=hist_model,
                            gaussianMixtureNoiseModel=gmm_model,
                            device='cpu')
data_dict['gmm']['x'][data_dict['gmm']['p'].argmax()]

In [ ]:
params = gmm_model.getGaussianParameters(signalBinIndex)
np.sqrt(np.sum((np.array(params[-6:])) * np.array(params[6:12])**2))

In [ ]:
# for i in range(histdata.shape[1]):
#     assert np.std(histdata[1][i]) < 1e-7
#     assert np.std(histdata[2][i]) < 1e-7

In [ ]:
# bin_val = (histdata[1] + histdata[2])/2
# bin_val = bin_val[:,0]
# binsize = np.mean(histdata[2] - histdata[1])
# bin_pdf = histdata[0]/binsize

In [ ]:
# from scipy.optimize import curve_fit
# import math
# import numpy as np

# def gaus(x, mu,sigma):
#     out = np.exp(-(x-mu)**2/(2*sigma**2)) * 1/(sigma*np.sqrt(2*math.pi))
#     # print(out.shape, out.min(), out.max())
#     return out

# def sigmoid(x):
#   return 1 / (1 + math.exp(-x))

In [ ]:
# index = 90
# x = bin_val
# y = bin_pdf[index]

# mean =bin_val[index]
# sigma = sum(y*(x-mean)**2)/len(y)

# popt,pcov = curve_fit(gaus,
#                       x,
#                       y,
#                       p0=[x[index],sigma])


In [ ]:
# pcov

In [ ]:
# popt

In [ ]:
# plt.plot(bin_val,bin_pdf[index],'b+:',label='data')
# plt.plot(bin_val,gaus(bin_val,*popt),'ro:',label='fit')
# plt.legend()
# plt.title('Fig. 3 - Fit for Time Constant')
# plt.xlabel('Time (s)')
# plt.ylabel('Voltage (V)')
# plt.show()

In [ ]:

from denoisplit.nets.hist_gmm_noise_model import HistGMMNoiseModel

In [ ]:
nm = HistGMMNoiseModel(histdata)
nm.fit()

In [ ]:

max_signal = hist_model.maxv.item()
min_signal = hist_model.minv.item()
n_bin = int(hist_model.bins.item())

histBinSize = (max_signal - min_signal) / n_bin
querySignal_numpy = (signalBinIndex / float(n_bin) * (max_signal - min_signal) + min_signal)


In [ ]:
nm._params = nm._params.cpu()

In [ ]:
signalBinIndex = 23
data_dict = plotProbabilityDistribution(signalBinIndex=signalBinIndex, 
                            histogramNoiseModel=hist_model,
                            gaussianMixtureNoiseModel=nm,
                            device='cpu')
data_dict['gmm']['x'][data_dict['gmm']['p'].argmax()]

In [ ]:
nm._min_valid_index

In [ ]:
nm._params[42]

In [ ]:
nm._binsize